In [ ]:
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD 
from tensorflow.keras.metrics import categorical_crossentropy 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os 
import cv2
import random
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from keras.layers import GlobalAveragePooling2D

In [ ]:
Dir = "input/vehicle-type-recognition/Dataset"

In [ ]:
labels = ['Bus', 'Car','Truck', 'motorcycle', ]

In [ ]:
data = []

In [ ]:
files = os.listdir(Dir)

for file in files:
    path= os.path.join(Dir,file)
    label = labels.index(file)
    for file in os.listdir(path):
        img_path = os.path.join(path , file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        data.append([img,label])  

In [ ]:
random.shuffle(data)
x= [] 
y = []
for features,label in data:
    x.append(features)
    y.append(label)
x = np.array(x)
y = np.array(y)

In [ ]:
img = x[7]
label = y[7]
    
plt.figure()
plt.imshow(img, cmap='gray')    # Print the image data
print("Label:", label)  # Print the label

In [ ]:
xyz = np.array([1 , 2, 3,0, 2, 3,1
               ])
xyz = to_categorical(xyz)

In [ ]:
xyz

In [ ]:
y= to_categorical(y)

In [ ]:
x_train , x_test , y_train, y_test = train_test_split( x , y , random_state= 0 , test_size= 0.2)

In [ ]:
unique_values, counts = np.unique(y_train, return_counts=True)

# Combine unique values and their counts
unique_counts = dict(zip(unique_values, counts))

# Print the unique values and their counts
for value, count in unique_counts.items():
    print(f"{value}: {count}")

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.3,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   brightness_range=[0.4, 1.5],
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Generate batches of augmented data from the training array
training_set = train_datagen.flow(x_train, y_train,
                                  batch_size=32,
                                  shuffle=True)

# Generate batches of augmented data from the testing array
test_set = test_datagen.flow(x_test, y_test,
                             batch_size=32,
                             shuffle=False)

In [ ]:
len(training_set)

In [ ]:
# Initialize MobileNet model
model = MobileNet(
    input_shape= (224,224,3),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)

In [ ]:
for layer in model.layers:
    layer.trainable=False

In [ ]:
final_model = Sequential()

# Add the MobileNet base model
final_model.add(model)
final_model.add(Flatten())
# Add the final dense layer for classification
final_model.add(Dense(4, activation='softmax'))

final_model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
checkpoint = ModelCheckpoint("Mobilenet.h5",monitor = "val_accuracy",save_best_only = True,verbose=1)
earlystop = EarlyStopping(monitor="val_accuracy",patience=8,verbose=1)

In [ ]:
y_train

In [ ]:
final_model.compile( optimizer= 'Adam' , loss ='categorical_crossentropy' , metrics= ['accuracy'])
#metrics=[ metrics.MeanSquaredError(), metrics.AUC()] 
#final_model.fit(x_train , y_train , epochs= 5 , validation_data=( x_test , y_test))
batch_size = 32  
History=final_model.fit(training_set, epochs=5, callbacks=[checkpoint,earlystop],validation_data=(test_set))